과제에서 배운 내용 정리

* 훈련 샘플에 순서에 민감한 학습 알고리즘이 있음
  * 비슷한 샘플이 연이어 나타나면 성능이 나빠지는 경우 데이터셋을 섞으면 됨
  * 섞어야 하는 대표적인 경우: SGD
    * 기본적으로 에포크 max_iter마다 훈련 데이터를 다시 섞음
  * 섞지 않아야 하는 대표적인 경우:시계열 데이터
* 사이킷런의 SGDClassifier
(Stochastic Gradient Descent 확률적 경사 하강법)
  * 장점: 매우 큰 데이터셋을 효율적으로 처리
bc 한 번에 하나씩 훈련 샘플을 독립적으로 처리
thus 온라인 학습에 잘 맞음
* 정확도를 분류기의 성능 측정 지표로 선호하지 않음, 특히 불균형한 데이터셋을 다룰 때 더욱

---

세션에서 배운 내용 정리
* scipy.ndimage.shift: https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.shift.html
  * shift(이미지, (y변화량, x변화량))
  * mode = constant (디폴트)
* 데이터 증식 후 shuffle을 해야 하나?
  * append를 이용해서 증식했기 때문에 shift된 이미지들이 같이 묶여있음
  * shuffle 해야하지만 KNN 알고리즘은 전체 샘플을 스캔해서 nearest neighbor를 찾기 때문에 샘플의 순서의 영향을 받지 않으므로 shuffle 하던 말던 무관함
* MNIST는 분류되는 숫자가 10개니까 KNN neighbors가 10이 적합하지 않을까?
  * Not necessarily: 분류되는 클래수의 수가 아니다, 가까운 주변 k개의 데이터를 기반으로 분류한다는 것.
  * https://bkshin.tistory.com/entry/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-6-K-%EC%B5%9C%EA%B7%BC%EC%A0%91%EC%9D%B4%EC%9B%83KNN
    * K의 default 값은 5입니다. 가장 가까운 주변 5개 데이터를 기반으로 분류한다는 것
    * 일반적으로 K는 홀수를 사용. 짝수일 경우 동점이 되어 하나의 결과를 도출할 수 없기 때문.
* grid_search.fit 할 때 execution 시간을 줄이기 위해 train 데이터셋의 일부만 써서 best_params_을 찾아도 괜찮은가? train 데이터 전체를 쓰는 것과 찾는 best_params_ 값이 다른가?
  * 똑같은 결과가 나왔음

---

링크

https://github.com/jparkgit/ESAA-24-1/blob/main/Week1_Session_Classification_Exercise.ipynb


## **| 분류 연습 문제**
___
출처 : 핸즈온 머신러닝 Ch03 분류 연습문제 1, 2번

In [1]:
# import data
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version = 1, as_frame = False)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
X, y = mnist["data"], mnist["target"]

In [3]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

### **1. MNIST 데이터셋으로 분류기를 만들어 테스트 세트에서 97% 정확도를 달성해보세요.**
___

1. `KNeghtborsClassifier`를 사용하는 것을 추천합니다.
2. `weights`와 `n_neighbors` 하이퍼 파라미터로 그리드 탐색을 시도하여, 좋은 하이퍼 파라미터 값을 찾아보세요.

In [4]:
# import package
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [10]:
X_train.shape

(60000, 784)

In [21]:
param_grid = [{'weights': ["uniform", "distance"],
               'n_neighbors': [3, 4, 5, 6, 7, 8, 9, 10]}]

knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, cv=5, scoring ="accuracy")
grid_search.fit(X_train[:10000], y_train[:10000])# Try GridSearch to optimize hyperparameter

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid=[{'n_neighbors': [3, 4, 5, 6, 7, 8, 9, 10],
                          'weights': ['uniform', 'distance']}],
             scoring='accuracy')

In [22]:
# best hyperparameter
grid_search.best_params_

{'n_neighbors': 4, 'weights': 'distance'}

In [ ]:
grid_search.score(X_test, y_test)

0.9714

In [10]:
knn_clf = KNeighborsClassifier(**grid_search.best_params_)

In [11]:
knn_clf.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=4, weights='distance')

In [12]:
knn_clf.score(X_test, y_test)

0.9714

n_neighbors uptil 9, using entire train dataset

In [24]:
param_grid = [{'weights': ["distance"],
    #'weights': ["uniform", "distance"],
               'n_neighbors': [3,5,7,9]}]

knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, cv=5, scoring ="accuracy")
grid_search.fit(X_train, y_train)# Try GridSearch to optimize hyperparameter
grid_search.best_score_

0.9711166666666665

In [26]:
# best hyperparameter
grid_search.best_params_

{'n_neighbors': 3, 'weights': 'distance'}

n_neighbors uptil 9, using 1/6 of train dataset

In [28]:
param_grid = [{'weights': ["distance"],
    #'weights': ["uniform", "distance"],
               'n_neighbors': [3,5,7,9]}]

knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, cv=5, scoring ="accuracy")
grid_search.fit(X_train[:10000], y_train[:10000])
grid_search.best_score_

0.9423999999999999

In [29]:
# best hyperparameter
grid_search.best_params_

{'n_neighbors': 3, 'weights': 'distance'}

In [ ]:
grid_search.score(X_test, y_test)

0.9714

In [25]:
# model test
# best_estimator 전체 데이터로 fit
grid_search.best_estimator_.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3, weights='distance')

In [27]:
grid_search.score(X_test, y_test)

0.9717

just a bit better

### **2. 다음 단계를 따라 인위적으로 훈련 세트를 늘리는 데이터 증식 또는 훈련 세트 확장 기법을 연습해봅시다.**
___

#### **STEP 1. MNIST 이미지를 (왼, 오른, 위, 아래) 어느 방향으로든 한 픽셀 이동시킬 수 있는 함수를 만들어 보세요.**

https://towardsdatascience.com/improving-accuracy-on-mnist-using-data-augmentation-b5c38eb5a903

https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.shift.html


Shift the image vertically by 20 pixels.

```
shift(image, (20, 0))
```

Shift the image vertically by -200 pixels and horizontally by 100 pixels.
```
shift(image, (-200, 100))
```

Shift the image vertically by dy pixels and horizontally by dx pixels.
```
shift(image, (dy, dx))
```


파라미터
* mode: {‘reflect’, ‘grid-mirror’, ‘constant’, ‘grid-constant’, ‘nearest’, ‘mirror’, ‘grid-wrap’, ‘wrap’}, optional

The mode parameter determines how the input array is extended beyond its boundaries. Default is ‘constant’.

* cval: scalar, optional

Value to fill past edges of input if mode is ‘constant’. Default is 0.0.

In [13]:
from scipy.ndimage import shift

def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    #shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    shifted_image = shift(image, [dy, dx])
    return shifted_image.reshape([-1])

Shuffle?

####  **STEP 2. 앞에서 만든 함수를 이용하여, 훈련 세트에 있는 각 이미지에 대해 네 개의 이동된 복사본(방향마다 한 개씩)을 만들어 훈련 세트에 추가하세요**

In [14]:
# Creating Augmented Dataset
X_train_augmented = [image for image in X_train]
y_train_augmented = [image for image in y_train]

for dx, dy in ((1,0), (-1,0), (0,1), (0,-1)):
     for image, label in zip(X_train, y_train):
             X_train_augmented.append(shift_image(image, dx, dy))
             y_train_augmented.append(label)

In [ ]:
X_train.shape

(60000, 784)

In [22]:
len(X_train_augmented)

300000

60,000 * 5 = 300,000\
(원본, 동, 서, 남, 북)

####  **STEP 3. 위에서 확장한 데이터셋을 이용하여, 1번 문제에서 찾은 최적 모델을 훈련시키고, 테스트 세트에서 정확도를 측정해보세요**

In [15]:
knn_clf2 = KNeighborsClassifier(**grid_search.best_params_)

In [16]:
knn_clf2.fit(X_train_augmented, y_train_augmented)

KNeighborsClassifier(n_neighbors=4, weights='distance')

In [17]:
knn_clf2.score(X_test, y_test)

0.9763

#### Shuffled?

In [18]:
# Shuffle the dataset
import numpy as np
shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented_shuffled = np.array(X_train_augmented)[shuffle_idx]
y_train_augmented_shuffled = np.array(y_train_augmented)[shuffle_idx]

In [19]:
knn_clf3 = KNeighborsClassifier(**grid_search.best_params_)
knn_clf3.fit(X_train_augmented_shuffled, y_train_augmented_shuffled)

KNeighborsClassifier(n_neighbors=4, weights='distance')

In [20]:
knn_clf3.score(X_test, y_test)

0.9763

The result is the same: [[출처]](https://datascience.stackexchange.com/questions/87142/does-shuffling-the-training-data-matter-in-a-k-nearest-neighbors-classifier-mode) if you are talking about K-nn algorithm per se, then, you do not have any advantage in shuffling your training set. Indeed, as you said, K-nn is just looking for the K nearest neighbors and does not care at all about the order of the samples, the algorithm will scan the entire training set for a single prediction, no matter what (unless you are using some efficient implementation like ball-tree or kd-tree).

참고한 사이트는 랜덤포레스트 써서 shuffle 한 걸로 보임